## Preparation

In [1]:
%%capture
!conda install -c pykaldi pykaldi -y

In [2]:
%cd /tmp

/tmp


In [3]:
!git clone https://github.com/pytorch/fairseq/

Cloning into 'fairseq'...
remote: Enumerating objects: 28172, done.
remote: Counting objects: 100% (586/586), done.
remote: Compressing objects: 100% (340/340), done.
remote: Total 28172 (delta 308), reused 464 (delta 233), pack-reused 27586
Receiving objects: 100% (28172/28172), 11.89 MiB | 18.37 MiB/s, done.
Resolving deltas: 100% (21133/21133), done.


In [4]:
!git clone https://github.com/kpu/kenlm

Cloning into 'kenlm'...
remote: Enumerating objects: 13824, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 13824 (delta 76), reused 92 (delta 45), pack-reused 13687
Receiving objects: 100% (13824/13824), 5.49 MiB | 10.15 MiB/s, done.
Resolving deltas: 100% (7956/7956), done.


In [5]:
%%capture
!apt-get -y install libeigen3-dev liblzma-dev zlib1g-dev libbz2-dev

In [6]:
%%capture
%cd kenlm
!mkdir build
%cd build
!cmake ..
!make -j 4
%cd /tmp

In [7]:
import os
os.environ['PATH'] = f"{os.environ['PATH']}:/tmp/kenlm/build/bin/"
os.environ['FAIRSEQ_ROOT'] = '/tmp/fairseq'

In [8]:
%cd /tmp/fairseq/

/tmp/fairseq


In [9]:
!pip install .

Processing /tmp/fairseq
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... - \ | / - done
    Preparing wheel metadata ... - \ done
     |████████████████████████████████| 123 kB 1.5 MB/s 
     |████████████████████████████████| 54 kB 1.6 MB/s 
     |████████████████████████████████| 112 kB 5.9 MB/s 
  Created wheel for fairseq: filename=fairseq-1.0.0a0+19793a7-cp37-cp37m-linux_x86_64.whl size=2041517 sha256=d05abf0e3fca30720b82126cd58dd5f54a6d2852498886b161f73f97c27111bf
  Stored in directory: /tmp/pip-ephem-wheel-cache-gbpk_myt/wheels/e3/52/72/c9070e4abac84e3448568d15e13e9e42bb689c08614271d8f3
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=7ce0b8f2536d8e21cda6a89e8f85c355be06d0ce16caf260c45b59923f491c99
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824bae

In [10]:
%cd /tmp/fairseq/examples/wav2vec/unsupervised

/tmp/fairseq/examples/wav2vec/unsupervised


In [11]:
os.environ['HYDRA_FULL_ERROR'] = '1'

## GAN

In [12]:
!ls /kaggle/input/wav2vec-u-cv-swedish-audio/

CLUS128		    precompute_pca512_cls128_mean	  train.npy
__notebook__.ipynb  precompute_pca512_cls128_mean_pooled  train.phn
__output__.json     test.lengths			  train.tsv
__results__.html    test.ltr				  train.wrd
custom.css	    test.npy				  valid.lengths
dict.test	    test.phn				  valid.ltr
dict.train	    test.tsv				  valid.npy
dict.valid	    test.wrd				  valid.phn
pca		    train.lengths			  valid.tsv
precompute_pca512   train.ltr				  valid.wrd


In [13]:
%%writefile rungan.sh
PREFIX=w2v_unsup_gan_xp
#TASK_DATA=/path/to/features/unfiltered/precompute_unfiltered_pca512_cls128_mean_pooled
TASK_DATA=/kaggle/input/wav2vec-u-cv-swedish-audio/precompute_pca512_cls128_mean_pooled/
#TEXT_DATA=/path/to/data  # path to fairseq-preprocessed GAN data
TEXT_DATA=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/
#KENLM_PATH=/path/to/data/kenlm.phn.o4.bin  # KenLM 4-gram phoneme language model (LM data = GAN data here)
KENLM_PATH=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/lm.phones.filtered.04.bin

PREFIX=$PREFIX fairseq-hydra-train \
	-m --config-dir /tmp/fairseq/examples/wav2vec/unsupervised \
	--config-path examples/wav2vec/unsupervised/config/gan \
	--config-name w2vu \
	task._name='unpaired_audio_text' \
	task.data=${TASK_DATA} \
	task.text_data=${TEXT_DATA} \
	task.kenlm_path=${KENLM_PATH} \
	checkpoint.no_epoch_checkpoints=false \
	checkpoint.save_dir=/kaggle/working \
	'common.seed=range(0,5)'

Writing rungan.sh


In [14]:
!bash rungan.sh

Traceback (most recent call last):
  File "/opt/conda/bin/fairseq-hydra-train", line 8, in <module>
    sys.exit(cli_main())
  File "/opt/conda/lib/python3.7/site-packages/fairseq_cli/hydra_train.py", line 76, in cli_main
    hydra_main()
  File "/opt/conda/lib/python3.7/site-packages/hydra/main.py", line 37, in decorated_main
    strict=strict,
  File "/opt/conda/lib/python3.7/site-packages/hydra/_internal/utils.py", line 355, in _run_hydra
    lambda: hydra.multirun(
  File "/opt/conda/lib/python3.7/site-packages/hydra/_internal/utils.py", line 201, in run_and_report
    raise ex
  File "/opt/conda/lib/python3.7/site-packages/hydra/_internal/utils.py", line 198, in run_and_report
    return func()
  File "/opt/conda/lib/python3.7/site-packages/hydra/_internal/utils.py", line 358, in <lambda>
    overrides=args.overrides,
  File "/opt/conda/lib/python3.7/site-packages/hydra/_internal/hydra.py", line 128, in multirun
    run_mode=RunMode.MULTIRUN,
  File "/opt/conda/lib/python3.7/site-